# Storing skymaps

## Read raw skymap files

In [3]:
import pickle
from pathlib import Path
import math
import numpy as np
from lsst.sphgeom import Box, ConvexPolygon, LonLat, UnitVector3d
import yaml
from skymap_to_poly_coords import load_pickle_skymap


In [4]:
package_root = Path.home() / "skymap-to-poly-coords"

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

skymap_out_dir = package_root / "skymaps_out"
inner_poly_path = skymap_out_dir / "inner_polygons.yaml"
outer_poly_path = skymap_out_dir / "outer_polygons.yaml"

In [5]:
lsst_skymap = load_pickle_skymap(skymap_path)
lsst_skymap

## A note on rings sky map pixelization 
*From the RingsSkyMap docstring in lsst.skymap:*

We divide the sphere into N rings of Declination, plus the two polar
caps, which sets the size of the individual tracts.  The rings are
divided in RA into an integral number of tracts of this size; this
division is made at the Declination closest to zero so as to ensure
full overlap.

Rings are numbered in the rings from south to north. The south pole cap is
``tract=0``, then the tract at ``raStart`` in the southernmost ring is
``tract=1``. Numbering continues (in the positive RA direction) around that
ring and then continues in the same fashion with the next ring north, and
so on until all reaching the north pole cap, which is
``tract=len(skymap) - 1``.

However, ``version=0`` had a bug in the numbering of the tracts: the first
and last tracts in the first (southernmost) ring were identical, and the
first tract in the last (northernmost) ring was missing. When using
``version=0``, these tracts remain missing in order to preserve the
numbering scheme.--->

## Storage options


We would like to support both inner and outer polygons for the tracts.  

Tracts are arranged in "rings", which span horizontal regions of the sky.

→ Inner polys:
- represent the exact boundaries of a given tract
- do not overlap

→ Outer polys:
- may (will?) overlap with adjacent tracts' outer polys
- are akin to pixels + margins in HATS catalogs

## Option 1: Two YAML files (inner polys and outer polys) that store each tracts' corners

Basically, a brute force approach where we calculate each tracts' vertices in radec and dump them into a yaml.

#### Inner polys/outer polys:
- Store per-tract: `ra_min`, `ra_max`, `dec_min`, `dec_max`
- Comes directly from `getRaDecRange(tract_index)`
- Requires 4 floats per tract, so for ~19k tracts:
  4 × 8 B × 19,000 ≈ ~600 KB (before YAML overhead; ~1–1.5 MB total)
- However, could compress to a `.npz` of around ~0.5–0.8 MB
- **Just kidding. Estimates were a bit off; a single file is about 4 MiB**

#### Pros
- Easy to understand and verify
- Doesn’t require any LSST WCS machinery
- Reading is fast + LSST-free

#### Cons
- Disk usage
- Less precise because of floating points?
- Storing patches is gonna go crazy on this. Better to iterate to a more performant version for that.
  - Otherwise, we'll have to break these files up into a series of yamls to accomodate patches.

### Write inner_poly and outer_poly files

In [4]:
from skymap_to_poly_coords import write_polygons_ra_dec


write_polygons_ra_dec(lsst_skymap, inner_poly_path, inner=True)
write_polygons_ra_dec(lsst_skymap, outer_poly_path, inner=False)

## Option 2: Reconstruct everything using projection + geometry

Can we do this without using LSST geom and mucking around with WCS packages and some really heavy projection/reprojection code?

I don't think re-implementing LSST's skymap package is in scope for this task, but open to suggestions if someone has an idea.

## Option 3: Hybrid; store per-ring Dec data, and per-tract RA data
### Similar principle with patches--each row of patches will act as its own "patch ring" with shared Dec

Can we make this assumption, or does it break down with projection distortions near the poles, rounding errors, or other confounding factors?

### Let's confirm two things:
1. Are the dec_min and dec_max values exactly the same for all tracts in a given ring?
2. Can each tract polygon be reconstructed from just the ring’s declination bounds and the tract’s RA vertex list?

### Assumption 1: dec_min and dec_max are exactly the same for all tracts in a ring (✓)
(within floating point imprecision)

In [6]:
import yaml
from pathlib import Path

yaml_path = inner_poly_path  # todo check outers also

# Load YAML
with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

# Parse and sort tracts
tract_items = sorted(data["tracts"].items(), key=lambda x: int(x[0]))

rings = []
current_ring = []
last_dec_bounds = None

for tract_id_str, entry in tract_items:
    tract_id = int(tract_id_str)
    polygon = entry.get("polygon")

    if polygon is None:
        print(f"Skipping degenerate tract {tract_id} with no polygon data.")
        continue

    decs = [point[1] for point in polygon]

    # Collect rounded decs to eliminate fp imprecision
    unique_decs = {round(d, 12) for d in decs}
    if len(unique_decs) > 2:
        raise ValueError(f"⚠️ Tract {tract_id} has {len(unique_decs)} unique Decs: {sorted(unique_decs)}")

    # Generate bounds for declination
    dec_min = round(min(decs), 12)
    dec_max = round(max(decs), 12)
    dec_bounds = (dec_min, dec_max)

    if last_dec_bounds is None:
        # First ring
        last_dec_bounds = dec_bounds

    if dec_bounds != last_dec_bounds:
        # New ring starts
        rings.append((last_dec_bounds, current_ring))
        current_ring = [tract_id]
        last_dec_bounds = dec_bounds
    else:
        current_ring.append(tract_id)

# Append the last ring
if current_ring:
    rings.append((last_dec_bounds, current_ring))

# Report
verbose = False
if verbose:
    for i, (bounds, tracts) in enumerate(rings):
        print(f"→ Ring {i}: dec_min/max = {bounds}, tracts = {tracts[0]} to {tracts[-1]}, total = {len(tracts)}")
else:
    print(f"→ Found {len(rings)} rings with {sum(len(r[1]) for r in rings)} tracts.")

→ Found 122 rings with 18938 tracts.


### Assumption 2: Each tract polygon can be reconstructed from just the ring’s declination bounds and the tract’s RA vertex list

This assumption relies on a couple other questions:
1. How many unique RA values are there per-tract? Always just 2, or sometimes 3 or 4?
2. Is there a patern to what order the polygon vertices are listed? Is it, say, always the 
  upper-left and clockwise around?

#### 2.1 How many unique RA values are in a tract?

In [7]:
from collections import Counter

ra_counts = Counter()

for tract_id_str, entry in data["tracts"].items():
    polygon = entry.get("polygon")
    if polygon is None:
        continue

    ras = [round(point[0], 12) for point in polygon]
    unique_ras = set(ras)
    ra_counts[len(unique_ras)] += 1

print("RA value counts per tract:")
for count, num_tracts in sorted(ra_counts.items()):
    print(f"  {count} unique RA values: {num_tracts} tracts")


RA value counts per tract:
  1 unique RA values: 2 tracts
  2 unique RA values: 18904 tracts
  3 unique RA values: 32 tracts


#### 2.2 Do most tracts follow the same curve orientation? If so, is it clockwise or counter-clockwise?

In [1]:
import math

def unwrap_ra_sequence(ras):
    """Unwrap RA values to make them monotonic, accounting for 0/360 crossover."""
    unwrapped = [ras[0]]
    for ra in ras[1:]:
        prev = unwrapped[-1]
        if ra < prev - 180:
            ra += 360
        elif ra > prev + 180:
            ra -= 360
        unwrapped.append(ra)
    return unwrapped

def polygon_orientation_ra_dec(points):
    """Return 'CW' or 'CCW' for a polygon defined by RA/Dec points.

    Parameters
    ----------
    points : list of (float, float)
        List of (RA, Dec) pairs in degrees. Polygon is assumed to be closed
        (first and last point the same) or will be treated as such.

    Returns
    -------
    str
        "CW" for clockwise, "CCW" for counter-clockwise, "Err" if not enough points.
    """
    if len(points) < 3:
        return "ERR"

    ras, decs = zip(*points)
    ras_unwrapped = unwrap_ra_sequence(ras)
    unwrapped_points = list(zip(ras_unwrapped, decs))

    # Shoelace formula: sum over (x_i * y_{i+1} - x_{i+1} * y_i)
    area = 0.0
    n = len(unwrapped_points)
    for i in range(n):
        x0, y0 = unwrapped_points[i]
        x1, y1 = unwrapped_points[(i + 1) % n]
        area += (x0 * y1 - x1 * y0)

    orientation = "CCW" if area > 0 else "CW"
    return orientation


In [17]:
from collections import Counter

ordering_counter = Counter()
clockwise_tracts = []

for tract_id_str, entry in data["tracts"].items():
    polygon = entry.get("polygon")
    if polygon is None or len(polygon) != 4:
        continue

    order = polygon_orientation_ra_dec(polygon)
    ordering_counter[order] += 1

    if order == "CW":
        clockwise_tracts.append(int(tract_id_str))

print("Polygon vertex ordering:")
for k, v in ordering_counter.items():
    print(f"  {k}: {v} tracts")

print()
print(f"{len(clockwise_tracts)} tracts are clockwise: {', '.join(map(str, clockwise_tracts))}")


Polygon vertex ordering:
  CW: 2 tracts
  CCW: 18936 tracts

2 tracts are clockwise: 0, 18937


#### 2.3 Do most tracts list their vertices starting from a certain corner? Which corner is that?

In [18]:
def classify_starting_corner(polygon):
    """Classify the starting corner of a polygon based on unwrapped RA/Dec.

    Parameters
    ----------
    polygon : list of (float, float)
        List of (RA, Dec) pairs in degrees. Assumed to be in polygon vertex order.

    Returns
    -------
    str
        One of: "top-left", "top-right", "bottom-left", "bottom-right"
    """
    if len(polygon) < 3:
        raise ValueError("Need at least 3 vertices to classify corners")

    # Unwrap RAs to make comparisons safe
    ras, decs = zip(*polygon)
    unwrapped_ras = unwrap_ra_sequence(ras)
    start_ra, start_dec = unwrapped_ras[0], decs[0]

    min_ra, max_ra = min(unwrapped_ras), max(unwrapped_ras)
    min_dec, max_dec = min(decs), max(decs)

    horiz = "left" if abs(start_ra - min_ra) < 1e-6 else "right"
    vert = "top" if abs(start_dec - max_dec) < 1e-6 else "bottom"

    return f"{vert}-{horiz}"


In [19]:
starting_corner_counter = Counter()

for tract_id_str, entry in data["tracts"].items():
    polygon = entry.get("polygon")
    if polygon is None or len(polygon) != 4:
        continue

    corner = classify_starting_corner(polygon)
    starting_corner_counter[corner] += 1

print("Starting corner classification:")
for corner, count in starting_corner_counter.items():
    print(f"  {corner}: {count} tracts")

Starting corner classification:
  bottom-left: 18938 tracts


#### Things we now know about our input skymap:
- All tracts start listing their vertices at the **bottom left** corner
- Almost all tracts list their vertices in **counter-clockwise** order--except the poles, who nominally use clockwise order, but...
- Almost all tracts have only 2 unique RA values; however:
  - 32 tracts have 3 unique RA values
  - 2 tracts have only 1 unique RA value (these are likely the poles)

So, we will follow the same convention for how we list our RA values in our ring-optimized poly coord files:
- Always start with the RA of the **bottom left** corner
- In most cases: list the 2 unique RA values (following the bottom-left rule, this will be the left-most RA, then the right-most)
- When we have 3 unique RA values, simply list all 4 RA values that are present, in **counter-clockwise** order
- *The poles will be weird. Still not sure the best way to encode them.*

## Rewrite code from Option 1 to be ring-aware

In [ ]:
# def write_ring_optimized_polygons_ra_dec(skymap, output_path, inner=True, write_patches=False):
#     """Write tract polygons in RA/Dec format to a YAML file, optimized by only storing declination
#     bounds on a ring basis.

#     Parameters
#     ----------
#     skymap : `lsst.skymap.SkyMap`
#         The LSST SkyMap object.
#     output_path : str or Path
#         Destination path for the output YAML file.
#     inner : bool, optional
#         If True, write inner polygons. If False, write outer polygons. Default is True.
#     write_patches : bool, optional
#         If True, include patch polygons for each tract. Default is False.
#     """
#     import lsst.geom as geom
#     from lsst.sphgeom import LonLat, Box

#     out = {"tracts": {}}

#     for tract in skymap:
#         tract_id = tract.getId()
#         if inner:
#             poly = tract.inner_sky_region
#             if isinstance(poly, Box):
#                 poly = box_to_convex_polygon(poly)
#         else:
#             poly = tract.outer_sky_polygon

#         ra_dec_vertices = [[radec[0], radec[1]] for radec in map(unit_vector3d_to_radec, poly.getVertices())]
#         out["tracts"][tract_id] = {"polygon": ra_dec_vertices}

#     with open(output_path, "w") as f:
#         yaml.dump(out, f, sort_keys=False)


In [20]:
def write_ring_optimized_polygons_ra_dec(skymap, output_path, inner=True):
    """Write tract polygons in RA/Dec format to a YAML file, grouped by ring and optimized
    by only storing declination bounds per ring and RA values per tract.

    Parameters
    ----------
    skymap : lsst.skymap.SkyMap
        The LSST SkyMap object.
    output_path : str or Path
        Destination path for the output YAML file.
    inner : bool, optional
        If True, write inner polygons. If False, write outer polygons. Default is True.
    """
    import lsst.geom as geom
    from lsst.sphgeom import Box
    from skymap_to_poly_coords.geometry import unit_vector3d_to_radec, box_to_convex_polygon

    out = {"rings": []}
    ring_counts = skymap._ringNums
    ring_start_index = 0

    for ring_id, n_tracts in enumerate(ring_counts):
        ring_entry = {"dec_bounds": None, "tracts": []}

        for i in range(ring_start_index, ring_start_index + n_tracts):
            tract = skymap[i]
            tract_id = tract.getId()

            # Get polygon and convert to RA/Dec
            poly = tract.inner_sky_region if inner else tract.outer_sky_polygon
            if isinstance(poly, Box):
                poly = box_to_convex_polygon(poly)

            radec_poly = [unit_vector3d_to_radec(vec) for vec in poly.getVertices()]
            ra_list = [pt[0] for pt in radec_poly]
            dec_list = [pt[1] for pt in radec_poly]

            # Collect unique rounded Decs (safe against FP jitter)
            unique_decs = sorted({round(d, 12) for d in dec_list})
            if len(unique_decs) > 2:
                raise ValueError(f"⚠️ Tract {tract_id} has >2 unique Decs: {unique_decs}")

            if ring_entry["dec_bounds"] is None:
                ring_entry["dec_bounds"] = [min(unique_decs), max(unique_decs)]

            # Round RAs and unwrap around 360 deg
            rounded_ras = [round(ra % 360.0, 12) for ra in ra_list]
            unique_ras = sorted(set(rounded_ras))

            # Determine order (bottom-left to top-left for CCW; TODO if needed)
            # Always start from bottom-left corner's RA
            bottom_left_ra = round(radec_poly[0][0] % 360.0, 12)
            if len(unique_ras) == 1:
                tract_entry = {"id": tract_id, "ra": [bottom_left_ra], "type": "pole"}
            elif len(unique_ras) == 2:
                tract_entry = {"id": tract_id, "ra": unique_ras}
            elif len(unique_ras) in (3, 4):
                # Preserve CCW order, starting from bottom-left corner
                # Reorder to start from bottom-left RA
                start_idx = rounded_ras.index(bottom_left_ra)
                ccw_ras = [rounded_ras[(start_idx + j) % 4] for j in range(4)]
                tract_entry = {"id": tract_id, "ra": ccw_ras}
            else:
                raise ValueError(f"Tract {tract_id} has unexpected number of unique RAs: {unique_ras}")

            ring_entry["tracts"].append(tract_entry)

        out["rings"].append(ring_entry)
        ring_start_index += n_tracts

    with open(output_path, "w") as f:
        yaml.dump(out, f, sort_keys=False)

    print(f"✅ Ring-optimized polygons written to {output_path}")
